# Quantum SVM

## Importing

In [ ]:
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua.input import ClassificationInput
from qiskit.aqua import run_algorithm, QuantumInstance
from qiskit.aqua.algorithms import QSVM
import numpy as np
import matplotlib.pyplot as plt

## Loading Real Backend

In [ ]:
#setting up account 
from qiskit import IBMQ
account_token = '130e1800ed9a45ead7808e858b7c540efdcd5c80efe80212e2539c3c3df64388d381a330a8b100601442298621aa7df0af1bee4421f3fced3d31bf76aa8e915e'
login_address = 'https://api.quantum-computing.ibm.com/api/Hubs/ibm-q/Groups/open/Projects/main'
IBMQ.enable_account(account_token, login_address)

## Loading Iris Dataset

In [ ]:
from DatasetLoader import LoadDataset

## QSVM

In [ ]:
from qiskit.aqua.components.feature_maps import SecondOrderExpansion, FirstOrderExpansion, PauliExpansion, self_product
#from qiskit.aqua.components.multiclass_extensions import AllPairs

#feature_dim = 4 # Iris contains 4 features: sepal length, sepal width, petal length, petal width
#feature_map_depth = 2 # circuit depth of feature map

# Linear entanglement means nearest neighbour entanglement

# Making a list of feature maps to test in combination with all multiclass extensions


# Creating train and test sets from Iris dataset
#train_set, test_set = Iris(20,5)
#extra_train_set, extra_test_set = Iris(1,5)
#datapoints, class_to_label = split_dataset_to_data_and_labels(extra_test_set)




# making a list of qsvms using different multiclass extensions and feature maps               
#qsvms = []
#for f in range(0, len(feature_maps)):

#    *** NOTE ***
# Apparently, the __init__.py for multiclass extensions does not include the name of
# any of the three available classes: OneAgainstRest, ErrorCorrectingCode and AllPairs.
# It's impossible to import any of them from the library.
# I have tried reinstalled the package with pip install qiskit -e (experimental mode)
# and edited my qiskit\aqua\components\multiclass_extensions\__init__.py to make extension 
# objects work. It takes too much time so I'll just switch to jsons from now on. 
#
#    __all__ = ['MulticlassExtension',
#*add line  'OneAgainstRest'
#*add line  'ErrorCorrectingCode'
#*add line  'AllPairs'
#           'Estimator']
    
#    one_against_rest = OneAgainstRest() 

# Also, it seems that when using declarative approach for multiclass, parameters on backend and
# feature map are not allowed, and I have commented out related sections specifying them. 
# I'll try to figure out why.

# This part converts data from Iris method to the format required by QSVM.
features = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
train_set, test_set = LoadDataset('../dataset/Iris_training.csv', '../dataset/Iris_testing.csv', features, label='Species')

temp = [test_set[k] for k in test_set]
total_array = np.concatenate(temp)
algo_input = ClassificationInput(train_set, test_set, total_array)


In [ ]:
#this method runs aqua with all possible combination of feature maps and multi-class expansions from a given config
def run_aqua(algo_input, real_backend = False, nshots = 1024):
    # This json object sets the basic config for the QSVM algorithm object. 
    # Since programmatic object themselves are not yet fully implemented in Aqua, 
    # this is the only possible way to define algorithm objects.
    aqua_dict = {
        'problem': {'name': 'classification'},
        'algorithm': {'name': 'QSVM'},
        'backend': {},
        'multiclass_extension': {},
        'feature_map': {}
    }
    
    # This part defines backend
    if real_backend:
        aqua_dict['backend'] = {'provider': 'qiskit.IBMQ', 'name': 'ibmq_16_melbourne', 'shots': nshots}
    else:
        aqua_dict['backend'] =  {'provider': 'qiskit.BasicAer', 'name': 'qasm_simulator', 'shots': nshots}
        
    # list of all extensions and feature maps.
    extensions = [
       {'name': 'OneAgainstRest'},
       {'name': 'AllPairs'}, 
       {'name': 'ErrorCorrectingCode', 'code_size': 5}]

    feature_maps = [
        {'name': 'SecondOrderExpansion', 'depth': 2, 'entanglement': 'linear'},
        {'name': 'FirstOrderExpansion', 'depth': 2}]
    
    # looping over them
    for extension in extensions:
        for feature_map in feature_maps:
            aqua_dict['multiclass_extension'] = extension
            aqua_dict['feature_map'] = feature_map
            result = run_algorithm(aqua_dict, algo_input)
            print("\n----- Using multiclass extension: '{}' -----\n".format(extension['name']))
            print("\n----- Using feature map: '{}' -----\n".format(feature_map['name']))
            for k,v in result.items():
                print("'{}' : {}".format(k, v))


In [ ]:
run_aqua(algo_input)